# Strands Agents SDK Demo by Markus Bestehorn (bestem@amazon.com)

Welcome to this simple demo of the **Strands Agents SDK** - a simple-to-use, code-first framework for building agents.

This notebook will walk you through the basics of creating and using agents with Strands. I recommend running this notebook in a [virtual Python environment](https://docs.python.org/3/library/venv.html).

## What is Strands Agents?

Strands Agents is a framework that makes it easy to build AI agents in Python. It provides a simple API for creating agents that can understand and respond to questions, perform tasks, and interact with various tools and services.

## Prerequisites

Before we start, make sure you have the Strands Agents SDK installed.

In [ ]:
# Install Strands Agents SDK
# Uncomment the line below if you haven't installed it yet
!pip install strands-agents strands-agents-tools strands-agents-builder -q

## 0. Authentication for AWS
There are different options which models to use and for authentication. Details are [here](https://strandsagents.com/0.1.x/user-guide/concepts/model-providers/amazon-bedrock/).

In [ ]:
import boto3
from strands import Agent
from strands.models import BedrockModel

session = boto3.Session(
    # aws_access_key_id='your_access_key',
    # aws_secret_access_key='your_secret_key',
    # aws_session_token='your_session_token',  # If using temporary credentials
    # region_name='us-west-2',
    profile_name='default'  # Optional: Use a specific profile
)

# Create a Bedrock model with the custom session
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    boto_session=session
)

## 1. Creating Your First Agent

Let's start by creating a simple agent with default settings.

In [ ]:
# Create an agent with settings using the above Session and Bedrock Model Information
agent = Agent(model=bedrock_model)

print("Agent created successfully!")
print(f"Agent type: {type(agent).__name__}")

## 2. Asking Your Agent a Question

Now let's ask our agent a simple question to see how it responds.

In [ ]:
# Ask the agent a question
response = agent("Tell me about agentic AI")
print("Agent's response:")
print(response)

## 3. Creating an Agent with Custom Configuration

Let's create a more customized agent with specific settings and behavior.

In [ ]:
# Create an agent with custom configuration
custom_agent = Agent(
    system_prompt="You are a senior Python developer.",
    model=bedrock_model,
)

## 4. Using the Custom Agent

Let's test our custom coding assistant agent.

In [ ]:
# Ask the custom agent a coding question
coding_question = "How do I create a simple list comprehension in Python?"
coding_response = custom_agent(coding_question)

print(f"Question: {coding_question}")
print(f"\nCodeHelper's response:")
print(coding_response)

## 5. Conversation History

Strands agents can maintain conversation history, allowing for contextual conversations.

In [ ]:
# Start a conversation with context
print("Starting a contextual conversation...")

# First question
response1 = custom_agent("What is machine learning?")
print("Question 1: What is machine learning?")
print(f"Response 1: {response1}")

print("\n" + "="*50 + "\n")

# Follow-up question (should reference previous context)
response2 = custom_agent("Can you give me a simple Python example of it?")
print("Question 2: Can you give me a simple Python example of it?")
print(f"Response 2: {response2}")

## 6. Agent with Tools (Advanced)

Strands agents can be equipped with tools to perform specific tasks. Let's create an agent with some basic tools.

In [ ]:
from strands import Agent, tool
import datetime
import random

# Define some simple tools
@tool
def get_current_time():
    """Get the current date and time."""
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def generate_random_number(min_val:int=1, max_val:int=100):
    """Generate a random number between min_val and max_val. min_val defaults to 1, max_val defaults to 100"""
    return random.randint(min_val, max_val)

@tool
def calculate_factorial(n:int):
    """Calculate the factorial of a number."""
    if n < 0:
        return "Factorial is not defined for negative numbers"
    elif n == 0 or n == 1:
        return 1
    else:
        result = 1
        for i in range(2, n + 1):
            result *= i
        return result

# Create an agent with tools
tool_agent = Agent(
    system_prompt="You are a tool equipped agent. Use them where appropriate.",
    tools=[
        get_current_time,
        generate_random_number,
        calculate_factorial
    ],
    model=bedrock_model,
)

print("Tool-equipped agent created!")


The important details in this piece of code are the following:

 - the @tool marker/decorator for the function indicates that this is a tool that agents can use. Without this decorator, the tools are not used even if they are passed to the Agent using the "tools" parameter.
 - The type hints are important. By default, agents are passed str variables. This means without the type hint "n:int" in the header of the function, agents would pass a value such as "5" (str) to the function. This will cause type errors or undesired behavior. Through the type hint, the agent knows that the value needs to be converted to int, e.g., "5" => int("5") => 5, before passing a value to this function.

 While agents sometimes can handle errors that occur when calling functions with wrongly formatted input, this will cause extra cycles for the inference as the agent first tries to use the tool, then waits for a response, discovers an errors, handles the error and then passes the input again to the function. Hence, providing proper type hints is highly recommended for tool definitions.   

## 7. Using Agent Tools

Now let's ask our tool-equipped agent to perform tasks using its tools.

In [ ]:
# Ask the agent to use its tools
questions_and_responses = [
    "What time is it right now?",
    "Can you generate a random number for me?",
    "What's the factorial of 5?",
    "Generate a random number between 50 and 100"
]

for question in questions_and_responses:
    response = tool_agent(question)
    print(f"Q: {question}")
    print(f"A: {response}")
    print("-" * 40)

There are several noteworthy details with the output and the corresponding code:

 - The 2nd question asks for a random number without specification of a range. The agent will use the defined defaults from the function in the code and generate a number between 1 and 100.
 - The 3rd question asks for the factorial. Normally, the function would be passed a str and not an int as required by the function. Through the use of Python type hints, one can provide the agent with information which format the input should have. In this case, the str is converted into int before it is passed to the factorial function.
 - The 4th question asks for random numbers with a range. Similar to the above, the type hints allow the agent to infer the right format for the input and use the tool correctly.

Without type hints, the agent would face difficulties to pass the input properly to the function. This can result in error messages such as:
~~~
I apologize for the error. It seems the function expects string values for the parameters...
~~~
In some cases, this can be resolved automatically, but to increase chances and speed of responses, using type hints is recommended.

## 8. Error Handling and Safety

Strands agents include built-in safety features and error handling.

In [ ]:
# Test error handling
try:
    # Try asking for something potentially problematic
    response = tool_agent("Calculate the factorial of -5")
    print("Response:", response)
except Exception as e:
    print(f"Error handled gracefully: {e}")

# Test with empty input
try:
    response = tool_agent("")
    print("Empty input response:", response)
except Exception as e:
    print(f"Empty input handled: {e}")

## 9. Agent Performance Monitoring

Let's look at some basic monitoring and evaluation of our agent's performance.

In [ ]:
import time

# Monitor response time
start_time = time.time()
response = agent("Explain what artificial intelligence is in simple terms")
end_time = time.time()

response_time = end_time - start_time
word_count = len(response.message["content"][0]["text"])

print(f"Response time: {response_time:.2f} seconds")
print(f"Response word count: {word_count} words")
print(f"Words per second: {word_count/response_time:.2f}")
# print("\nResponse:")
# print(response)

## Conclusion

This notebook has demonstrated the key features of the Strands Agents SDK:

1. **Simple Agent Creation**: Easy to create agents with minimal code
2. **Customization**: Configure agents with specific parameters and behavior
3. **Tool Integration**: Equip agents with custom tools and functions
4. **Conversation Context**: Maintain conversation history for contextual interactions
5. **Error Handling**: Built-in safety and error handling mechanisms
6. **Performance Monitoring**: Track and evaluate agent performance
7. **Production Readiness**: Configuration options for deployment

### Next Steps

To learn more about Strands Agents:
- Visit the [official documentation](https://strandsagents.com/0.1.x/)
- Explore the User Guide sections:
  - Welcome
  - Quickstart
  - Concepts
  - Safety & Security
  - Observability & Evaluation
  - Deploy

Happy building with Strands Agents! 🚀